## logistic regression model

#### import libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#### load cleaned feature data table

In [4]:
df = pd.read_csv('clean_ft_table.csv')

In [20]:
df[df['re_intub_class']==1]

,spontRR,stdABP,meanABP,stdSpontRR,pulseox,stdPulseox,temp,heartRate,stdHeartRate,weight,height,re_intub_class,anchor_age,time_on_vent,adm_type_ELECTIVE,adm_type_EW EMER.,adm_type_OBSERVATION ADMIT,adm_type_SURGICAL SAME DAY ADMISSION,adm_type_URGENT,gender_M
10637,17.0,6.260990,80.0,3.204164,95.0,2.280351,36.888889,81.0,5.750362,196.700000,70.039370,1,61,25.500000,0,0,1,0,0,1
10638,32.0,12.722683,106.0,4.855042,90.0,1.861899,37.166667,121.0,10.962056,88.950000,55.952756,1,58,93.116667,0,0,1,0,0,0
10639,20.0,5.167204,59.0,1.516575,99.0,1.095445,36.222222,81.0,6.580274,166.100000,61.011811,1,85,29.833333,0,0,0,0,1,0
10640,15.0,10.788883,93.0,3.082207,98.0,0.816497,37.666667,101.0,11.545562,156.860000,68.055118,1,42,100.500000,0,0,1,0,0,1
10641,12.0,22.793182,74.0,6.209152,97.0,2.632835,36.900000,53.0,3.869069,193.600000,73.561024,1,61,6.150000,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11633,22.0,5.902481,84.0,2.326094,100.0,0.000000,37.722222,75.0,3.352327,189.200000,70.039370,1,65,14.783333,0,1,0,0,0,1
11634,13.0,3.761299,64.5,1.863525,96.0,1.354006,37.055556,94.0,2.321398,139.000000,61.011811,1,65,73.466667,0,0,0,0,0,0
11635,17.0,12.378938,71.0,1.772811,100.0,0.000000,36.888889,80.0,2.483277,160.600000,64.086614,1,53,218.083333,0,1,0,0,0,0
11636,21.0,296.433466,73.0,8.952254,100.0,1.549193,36.888889,71.0,14.052283,236.133333,70.039370,1,72,98.900000,0,1,0,0,0,1


## Train model

### perform train test split

In [35]:
X = df[df.columns.drop('re_intub_class')]
y = df['re_intub_class']

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#### 2. Perform feature scaling

Because the range of values in the features are not necessarily in the same order of magnitude, we will scale the feature data prior to training the model.

* actually... they might not be far off! 

In [37]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [38]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#### 3. SMOTE it!

In [43]:
from collections import Counter
counter = Counter(y_train)
print(counter)

Counter({0: 7423, 1: 723})


In [40]:
import imblearn
print(imblearn.__version__)

0.7.0


In [41]:
from imblearn.over_sampling import SMOTE

In [42]:
# transform the dataset
oversample = SMOTE(random_state = 101)

In [44]:
X_smote, y_smote = oversample.fit_resample(X_train, y_train)

In [45]:
counter = Counter(y_smote)
print(counter)

Counter({0: 7423, 1: 7423})


#### 4. Do logistic regression model

In [46]:
from sklearn.linear_model import LogisticRegression

In [47]:
logmodel = LogisticRegression()
logmodel.fit(X_smote,y_smote)

LogisticRegression()

In [48]:
predictions = logmodel.predict(X_test)

#### 5. Evaulate model

In [49]:
from sklearn.metrics import classification_report, confusion_matrix

In [50]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.95      0.58      0.72      3214
           1       0.11      0.62      0.19       278

    accuracy                           0.58      3492
   macro avg       0.53      0.60      0.46      3492
weighted avg       0.88      0.58      0.68      3492



In [51]:
print(confusion_matrix(y_test,predictions))

[[1865 1349]
 [ 106  172]]
